In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import matplotlib.pyplot as plt

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import glob
import os.path as osp
from PIL import Image

import torchvision.datasets as dset
import torchvision.transforms as T
import chest_xray_code.data.xrays as preprocess_dataset
import chest_xray_code.data.raw_reports as utils
import os
import torch.nn.functional as F
from models.TestConvNet import TestConvNet 
from loaders.BloodCellLoader import BloodCellLoader




import numpy as np

In [3]:
trainset = BloodCellLoader(
    root='blood_cells_data/dataset-master/JPEGImages',
    preload=False, transform=transforms.ToTensor(),
)
# Use the torch dataloader to iterate through the dataset
trainset_loader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=24)

# load the testset
# testset = Data_SET(
#     root='chest_xray_code/data/xrays',
#     preload=True, transform=transforms.ToTensor(),
# )
#testset = trainset
# Use the torch dataloader to iterate through the dataset
#testset_loader = DataLoader(testset, batch_size=1000, shuffle=False, num_workers=1)

print(len(trainset))

366


In [4]:
transform = T.Compose([
                T.ToTensor()
                #T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])
images = {}

In [5]:
USE_GPU = True

dtype = torch.float32 

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    #dtype = torch.cuda.FloatTensor
else:
    device = torch.device('cpu')

In [5]:
channels = 3
size = 32

In [6]:
i = 0
images = []
for data in trainset_loader:
    with torch.no_grad():
        data = data.to(device)
        img = data.cpu().detach()
    
        for i in range(img.shape[0]):
            individual_img = img[i]
            individual_img = individual_img.numpy()
            individual_img = np.transpose(individual_img, (1, 2, 0))
            images.append(torch.from_numpy(individual_img))

In [7]:
loss_function = nn.MSELoss(size_average=True)
total_comparisons = 0
comparison_sum = 0
for i in range(len(images)):
    for j in range(len(images)):
        if i != j:
            comparison_sum += loss_function(images[i],images[j])
            total_comparisons += 1
homogeneity = comparison_sum / total_comparisons

In [8]:
print(homogeneity.item())

0.01453955564647913
